In [3]:
import itertools
import numpy as np
import nltk
from sklearn import(
    datasets, feature_extraction, model_selection, 
    pipeline, naive_bayes, metrics
)
import matplotlib.pyplot as plt

In [ ]:
def extract_features(corpus):
    '''Extract TF-IDF features from corpus'''
    stop_words = nltk.corpus.stopwords.words('english')
    count_vectorizer = feature_extraction.text.CountVectorizer(
        tokenizer=nltk.word_tokenize,
        stop_words=stop_words,
        min_df=2, # The word must appear more than once
        # The higher ngram range is, the higher the vector space and computing cost
        ngram_range=(1, 2), # Allows for 1 and 2 word combinations
    )
    processed_corpus = count_vectorizer.fit_transform(corpus)
    # Can change params of TfidfTransformer if it lowers performance
    processed_corpus = feature_extraction.text.TfidfTransformer(
        ).fit_transform(processed_corpus)
    
    return processed_corpus